# Step 1: Data Preprocessing for AI Vet Consultant

이 노트북에서는 AI 수의사 상담 시스템을 위한 데이터를 준비합니다.
CSV 파일에 있는 반려동물의 증상과 정보를 AI 모델이 이해하기 쉬운 **프롬프트 형식**으로 변환하는 과정을 실습합니다.

In [1]:
import pandas as pd
import os

# 데이터 경로 설정 (더 큰 데이터셋 사용)
INPUT_CSV_PATH = "../00_datasets/dr.tail/Dr.tail_210.CSV"
OUTPUT_DIR = "./data"

# 출력 디렉토리 생성
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)
    print(f"Created directory: {OUTPUT_DIR}")

## 1. 데이터 로드 및 확인
Pandas를 사용하여 CSV 파일을 읽고 데이터 구조를 확인합니다. 실습을 위해 5개만 샘플링합니다.

In [2]:
try:
    df_raw = pd.read_csv(INPUT_CSV_PATH)
    df = df_raw.head(5).copy()
    print(f"Successfully loaded dataset. Using first {len(df)} cases for exercise.")
    display(df.head(2))
except FileNotFoundError:
    print(f"Error: File not found at {INPUT_CSV_PATH}")

Successfully loaded dataset. Using first 5 cases for exercise.


,Case_ID,symptom,pet_information,issue_detailes,more_details,answer,Species
0,2,Diarrhea,Name\nLucyfur\nSpecies\nDog\nBreed\nPortuguese...,Main Concern\nDiarrhea\nColor\nGreen\nFrequenc...,She has had bad stinky gas for days. Eating gr...,The symptoms you're describing in your Portugu...,dog
1,3,Diarrhea,Name\nBow\nSpecies\nDog\nBreed\nPyrenean Sheph...,Main Concern\nDiarrhea\nColor\nBrown\nFrequenc...,"Hey, Like spray shits diarrhea. Every time he ...",Diarrhea in a 3-month-old Pyrenean Shepherd pu...,dog


## 2. 프롬프트 포맷팅 함수 작성 (실습)
각 행(Row)의 정보를 하나의 완성된 질문(Prompt) 텍스트로 합치는 함수를 작성해 봅시다.

**목표 형식:**
```
증상 (Symptom):
[증상 내용]

반려동물 정보 (Pet Information):
[반려동물 정보]

상세 문제 (Issue Details):
[상세 문제]

보호자 추가 설명 (More Details from Guardian):
[보호자 추가 설명]
```

In [4]:
def format_input_from_row(row):
    """Pandas 데이터프레임의 한 행을 받아 AI 입력 형식 문자열로 변환합니다."""
    
    # 1. 데이터 추출 (없는 경우 빈 문자열 처리)
    symptom = str(row.get('symptom', ''))
    pet_info = str(row.get('pet_information', ''))
    issue_details = str(row.get('issue_details', ''))
    more_details = str(row.get('more_details', ''))
    
    # 2. 포맷팅 (여기에 코드를 작성하세요)
    full_description = f"""
증상 (Symptom):
{symptom}

반려동물 정보 (Pet Information):
{pet_info}

상세 문제 (Issue Details):
{issue_details}

보호자 추가 설명 (More Details from Guardian):
{more_details}
"""
    return full_description.strip()

# 테스트: 첫 번째 행으로 결과 확인
if not df.empty:
    sample_row = df.iloc[0]
    print(format_input_from_row(sample_row))

증상 (Symptom):
Diarrhea

반려동물 정보 (Pet Information):
Name
Lucyfur
Species
Dog
Breed
Portuguese Podengo Pequeno
Age
3 years
Gender
Female
Status
Intact

상세 문제 (Issue Details):


보호자 추가 설명 (More Details from Guardian):
She has had bad stinky gas for days. Eating grass and puked.


## 3. 전체 데이터 변환 및 저장
모든 데이터에 이 함수를 적용하고, 나중에 사용할 수 있도록 저장합니다.

In [5]:
# 새로운 컬럼 'formatted_prompt' 생성
df['formatted_prompt'] = df.apply(format_input_from_row, axis=1)

# 필요한 컬럼만 선택하여 저장 (Prompt와 정답 Answer)
df_processed = df[['formatted_prompt', 'answer']]

output_path = os.path.join(OUTPUT_DIR, "processed_sample.csv")
df_processed.to_csv(output_path, index=False, encoding='utf-8-sig')

print(f"Saved {len(df_processed)} processed cases to {output_path}")
display(df_processed.head(1))

Saved 5 processed cases to ./data\processed_sample.csv


,formatted_prompt,answer
0,증상 (Symptom):\nDiarrhea\n\n반려동물 정보 (Pet Information):\nName\n...,The symptoms you're describing in your Portugu...
